In [ ]:
from pathlib import Path

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch import optim
from torch.nn import functional as F
from torch.optim.lr_scheduler import _LRScheduler
from torch.utils.data import TensorDataset, DataLoader

In [3]:
seed = 1
np.random.seed(seed)

In [5]:
DATA = Path.home()/'data'/'careercon2019'/'tmp'
raw_arr = np.load(DATA/'feat.npy').transpose(0, 2, 1)
fft_arr = np.load(DATA/'feat_fft.npy').transpose(0, 2, 1)
target = np.load(DATA/'target.npy')
print(f'''
Prepared datasets shapes:
{raw_arr.shape} raw
{fft_arr.shape} fft
{target.shape} target
''')


Prepared datasets shapes:
(7626, 9, 128) raw
(7626, 6, 65) fft
(7626,) target



In [ ]:
# Shape of array: (batch, features, time dimension)
# raw_arr = data.values.reshape([trn_sz + tst_sz, len(data.columns),  seq_len])
# fft_arr = fft_data.values.reshape([trn_sz + tst_sz, len(fft_data.columns), fft_seq_len])
# print(f'Prepared datasets shapes: {raw_arr.shape} raw, {fft_arr.shape} fft')

In [ ]:
# enc = LabelEncoder().fit(y_trn['surface'])
# target = list(enc.transform(y_trn['surface']))
# target += [0] * tst_sz
# target = np.array(target)
# assert len(target) == trn_sz + tst_sz

In [6]:
def create_datasets(data, target, train_size, valid_pct=0.1, seed=None):
    raw, fft = data
    assert len(raw) == len(fft)
    sz = train_size
    idx = np.arange(sz)
    trn_idx, val_idx = train_test_split(
        idx, test_size=valid_pct, random_state=seed)
    trn_ds = TensorDataset(
        torch.tensor(raw[:sz][trn_idx]).float(), 
        torch.tensor(fft[:sz][trn_idx]).float(), 
        torch.tensor(target[:sz][trn_idx]).long())
    val_ds = TensorDataset(
        torch.tensor(raw[:sz][val_idx]).float(), 
        torch.tensor(fft[:sz][val_idx]).float(), 
        torch.tensor(target[:sz][val_idx]).long())
    tst_ds = TensorDataset(
        torch.tensor(raw[sz:]).float(), 
        torch.tensor(fft[sz:]).float(), 
        torch.tensor(target[sz:]).long())
    return trn_ds, val_ds, tst_ds

In [7]:
def create_loaders(data, bs=128, jobs=0):
    trn_ds, val_ds, tst_ds = data
    trn_dl = DataLoader(trn_ds, batch_size=bs, shuffle=True, num_workers=jobs)
    val_dl = DataLoader(val_ds, batch_size=bs, shuffle=False, num_workers=jobs)
    tst_dl = DataLoader(tst_ds, batch_size=bs, shuffle=False, num_workers=jobs)
    return trn_dl, val_dl, tst_dl

In [9]:
trn_sz = 3810

In [10]:
datasets = create_datasets((raw_arr, fft_arr), target, trn_sz, seed=seed)

In [11]:
def bn1d_drop_layer(layers, n_outputs, drop=None, bn=True, activ=nn.ReLU):
    """Adds batchnorm, dropout, and/or activation layer(s) 
    to the list of layers.
    """
    if bn:
        layers += [nn.BatchNorm1d(n_outputs)]
    if activ is not None:
        layers += [activ()]
    if drop and 0.0 < drop < 1.0:
        layers += [nn.Dropout(drop)]
    return layers

In [12]:
def conv1d(ni, no, kernel=3, stride=1, pad=0,
           drop=None, bn=True, activ=nn.ReLU):
    """A 1-d convolutional layer with few additional layers on top of it."""
    
    layers = [nn.Conv1d(ni, no, kernel, stride, pad, bias=not bn)]
    return bn1d_drop_layer(layers, no, drop, bn, activ)

In [13]:
def fc(ni, no, drop=None, bn=True, activ=nn.ReLU):
    """A fully connected layer with few additional layers on top of it."""
    
    layers = [nn.Linear(ni, no, bias=not bn)]
    return bn1d_drop_layer(layers, no, drop, bn, activ)

In [15]:
class SeparableConv1d(nn.Module):
    def __init__(self, ni, no, kernel, stride, pad):
        super().__init__()
        self.depthwise = nn.Conv1d(ni, ni, kernel, stride, padding=pad, groups=ni)
        self.pointwise = nn.Conv1d(ni, no, kernel_size=1)

    def forward(self, x):
        return self.pointwise(self.depthwise(x))

In [16]:
class Flatten(nn.Module):
    """Converts N-dimensional tensor into 'flat' one."""

    def __init__(self, keep_batch_dim=True):
        super().__init__()
        self.keep_batch_dim = keep_batch_dim

    def forward(self, x):
        if self.keep_batch_dim:
            return x.view(x.size(0), -1)
        return x.view(-1)

In [32]:
class Classifier(nn.Module):
    def __init__(self, raw_ni, fft_ni, no):
        super().__init__()
        # 3, 2, 2, 2
        self.raw = nn.Sequential(
            SeparableConv1d(raw_ni, 32, 8, 2, 3),
            nn.ReLU(),
            nn.Dropout(.5),
            SeparableConv1d(32, 64, 8, 4, 2),
            nn.ReLU(),
            nn.Dropout(.5),
            SeparableConv1d(64, 128, 8, 4, 2),
            nn.ReLU(),
            nn.Dropout(.5),
            SeparableConv1d(128, 256, 8, 4, 2),
            nn.ReLU(),
            Flatten(),
            nn.Dropout(.5),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Dropout(.5),
            nn.Linear(64, 64),
            nn.ReLU()
        )
        # 4, 4, 4, 4, 3
        self.fft = nn.Sequential(
            SeparableConv1d(fft_ni, 32, 8, 2, 4),
            nn.ReLU(),
            nn.Dropout(.5),
            SeparableConv1d(32, 64, 8, 2, 4),
            nn.ReLU(),
            nn.Dropout(.5),
            SeparableConv1d(64, 128, 8, 4, 4),
            nn.ReLU(),
            nn.Dropout(.5),
            SeparableConv1d(128, 128, 8, 4, 4),
            nn.ReLU(),
            nn.Dropout(.5),
            SeparableConv1d(128, 256, 8, 2, 3),
            nn.ReLU(),
            Flatten(),
            nn.Dropout(.5),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Dropout(.5),
            nn.Linear(64, 64),
            nn.ReLU()
        )
        self.out = nn.Sequential(
            nn.Linear(128, 64),
            nn.ReLU(inplace=True),
            nn.Linear(64, no)
        )
    
    def forward(self, t_raw, t_fft):
        raw_out = self.raw(t_raw)
        fft_out = self.fft(t_fft)
        t_in = torch.cat([raw_out, fft_out], dim=1)
        out = self.out(t_in)
        return out

In [33]:
class CyclicLR(_LRScheduler):
    
    def __init__(self, optimizer, schedule, last_epoch=-1):
        assert callable(schedule)
        self.schedule = schedule
        super().__init__(optimizer, last_epoch)

    def get_lr(self):
        return [self.schedule(self.last_epoch, lr) for lr in self.base_lrs]

In [34]:
def cosine(t_max, eta_min=0):
    
    def scheduler(epoch, base_lr):
        t = epoch % t_max
        return eta_min + (base_lr - eta_min)*(1 + np.cos(np.pi*t/t_max))/2
    
    return scheduler

In [35]:
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

In [36]:
raw_feat = raw_arr.shape[1]
fft_feat = fft_arr.shape[1]

trn_dl, val_dl, tst_dl = create_loaders(datasets, bs=256)

lr = 0.001
n_epochs = 3000
iterations_per_epoch = len(trn_dl)
num_classes = 9
best_acc = 0
patience, trials = 500, 0

model = Classifier(raw_feat, fft_feat, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
opt = torch.optim.Adam(model.parameters(), lr=lr)
# sched = CyclicLR(opt, cosine(t_max=iterations_per_epoch * 2, eta_min=lr/100))

print('Start model training')

for epoch in range(1, n_epochs + 1):
    
    model.train()
    for i, batch in enumerate(trn_dl):
        x_raw, x_fft, y_batch = [t.to(device) for t in batch]
        # sched.step()
        opt.zero_grad()
        out = model(x_raw, x_fft)
        loss = criterion(out, y_batch)
        loss.backward()
        opt.step()
    
    model.eval()
    correct, total = 0, 0
    for batch in val_dl:
        x_raw, x_fft, y_batch = [t.to(device) for t in batch]
        out = model(x_raw, x_fft)
        preds = F.log_softmax(out, dim=1).argmax(dim=1)
        total += y_batch.size(0)
        correct += (preds == y_batch).sum().item()
    
    acc = correct / total

    if epoch % 5 == 0:
        print(f'Epoch: {epoch:3d}. Loss: {loss.item():.4f}. Acc.: {acc:2.2%}')

    if acc > best_acc:
        trials = 0
        best_acc = acc
        torch.save(model.state_dict(), 'best.pth')
        print(f'Epoch {epoch} best model saved with accuracy: {best_acc:2.2%}')
    else:
        trials += 1
        if trials >= patience:
            print(f'Early stopping on epoch {epoch}')
            break

Start model training
Epoch 1 best model saved with accuracy: 16.54%
Epoch 2 best model saved with accuracy: 19.69%
Epoch:   5. Loss: 2.0481. Acc.: 19.69%
Epoch:  10. Loss: 2.0159. Acc.: 19.69%
Epoch:  15. Loss: 1.8437. Acc.: 33.07%
Epoch 15 best model saved with accuracy: 33.07%
Epoch 18 best model saved with accuracy: 34.12%
Epoch:  20. Loss: 1.6236. Acc.: 35.17%
Epoch 20 best model saved with accuracy: 35.17%
Epoch 23 best model saved with accuracy: 35.70%
Epoch 24 best model saved with accuracy: 38.58%
Epoch:  25. Loss: 1.6183. Acc.: 38.06%
Epoch 26 best model saved with accuracy: 40.42%
Epoch 28 best model saved with accuracy: 40.68%
Epoch:  30. Loss: 1.3745. Acc.: 45.93%
Epoch 30 best model saved with accuracy: 45.93%
Epoch 31 best model saved with accuracy: 46.98%
Epoch 32 best model saved with accuracy: 50.66%
Epoch 33 best model saved with accuracy: 56.43%
Epoch:  35. Loss: 1.1143. Acc.: 54.86%
Epoch 36 best model saved with accuracy: 58.53%
Epoch 38 best model saved with accur

Epoch: 765. Loss: 0.5320. Acc.: 78.74%
Epoch: 770. Loss: 0.5284. Acc.: 79.00%
Epoch: 775. Loss: 0.4602. Acc.: 78.74%
Epoch: 780. Loss: 0.6031. Acc.: 77.43%
Epoch: 785. Loss: 0.4820. Acc.: 78.22%
Epoch: 790. Loss: 0.5692. Acc.: 79.27%
Epoch: 795. Loss: 0.4460. Acc.: 80.58%
Epoch: 800. Loss: 0.4375. Acc.: 77.95%
Epoch: 805. Loss: 0.5016. Acc.: 79.00%
Epoch: 810. Loss: 0.5946. Acc.: 77.95%
Epoch: 815. Loss: 0.4850. Acc.: 80.31%
Epoch: 820. Loss: 0.4481. Acc.: 78.74%
Epoch: 825. Loss: 0.4893. Acc.: 79.27%
Epoch: 830. Loss: 0.3550. Acc.: 78.74%
Epoch: 835. Loss: 0.4429. Acc.: 81.10%
Epoch: 840. Loss: 0.4308. Acc.: 79.53%
Epoch: 845. Loss: 0.4799. Acc.: 80.05%
Epoch: 850. Loss: 0.4074. Acc.: 80.58%
Epoch: 855. Loss: 0.5737. Acc.: 79.79%
Epoch: 860. Loss: 0.5978. Acc.: 79.79%
Epoch: 865. Loss: 0.4985. Acc.: 79.27%
Epoch: 870. Loss: 0.3784. Acc.: 79.27%
Epoch: 875. Loss: 0.3345. Acc.: 78.48%
Epoch: 880. Loss: 0.4849. Acc.: 79.27%
Epoch: 885. Loss: 0.3883. Acc.: 78.74%
Epoch: 890. Loss: 0.5482.

In [31]:
fft_arr.shape

(7626, 6, 65)